<h1><a href="https://wandb.ai/carsonstevens/Epic?workspace=user-carsonstevens">Weights and Biases Project Page</a></h1>

In [ ]:
import ipywidgets
from IPython import display
import os

!pip install transformers
!pip install wandb
import wandb


import torch
from transformers import TextDataset,DataCollatorForLanguageModeling
from transformers import GPT2Model, GPT2Config
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModel
from transformers import Trainer, TrainingArguments


display.clear_output()
from google.colab import drive
drive.mount('/content/drive')


display.clear_output()
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9

if gpu_info.find('failed') >= 0 and ram_gb < 30:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  try:
      if gpu_info.find('failed') < 0:
         print(gpu_info)
  except:
    display.clear_output()
  finally:
    print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
    print('You are using a high-RAM runtime!')


def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False, mlm_probability=0.15
    )
    return train_dataset,test_dataset,data_collator


    

Sat Nov 28 00:58:24 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 300W |     10MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
parent_directory = '/content/drive/My Drive/CSCI 470 Project/Epic/'
checkpoint_path = 'Models/gpt2_large_horror_generator/'
last_checkpoint = parent_directory+checkpoint_path+"checkpoint-"+'18000'
# wandb.init(config={"epochs": epochs, "batch_size": batch_size})

literary_tokens = {"characterization": '[CHARACTERIZATION]', "character": '[CHARACTER]', "setting": '[SETTING]', 
                   "exposition": '[EXPOSTION]', "climax": '[CLIMAX]', "resolution": '[RESOLUTION]', "plot": '[PLOT]', 
                   "context": '[CONTEXT]', "action": '[ACTION]', "weapon": '[WEAPON]', "danger": '[DANGER]', "death": '[DEATH]', "suspense": '[SUSPENSE]', 
                   "emotion": '[EMOTION]', "surprise": '[SURPRISE]', "problem":'[PROBLEM]', "conflict": '[CONFLICT]', 
                   "perspective": '[PERSPECTIVE]', "transition": '[TRANSITION}', "relief": '[RELIEF]', "metaphor": '[METAPHOR]', 
                   "flashback": '[FLASHBACK]'}

subgenre_tokens = {'Vampire': '[VAMPIRE]', 'Ghost' : '[GHOST]', 'Horror' : '[HORROR]', 'Comedic Horror' : '[COMEDIC HORROR]', 'Murder' : '[MURDER]', 'Werewolf' : '[WEREWOLF]', 'Apocalypse' : '[APOCALYPSE]',
              'Haunted House' : '[HAUNTED HOUSE]', 'Witch' : '[WITCH]', 'Hell' : '[HELL]', 'Alien' : '[ALIEN]', 'Gore' : '[GORE]', 'Monster' : '[MONSTER]'}


author_tokens = {'Clive Barker' : '[CLIVE BARKER]', 'J. K. Rowling' : '[J.K. ROWLING]', 'Stephen King' : '[STEPHEN KING]', 'Théophile Gautier' : '[THEOPHILE GAUTIER]', 
               'James H. Hyslop' : '[JAMES H HYSLOP]', 'Lord Edward Bulwer-Lytton' : '[LORD EDWARD BULWER-LYTTON]', 'A. T. Quiller-Couch' : '[A. T. QUILLER-COUCH]', 
               'Mrs. Margaret Oliphant' : '[MRS. MARGARET OLIPHANT]', 'Ernest Theodor Amadeus Hoffmann' : '[ERNEST THEODOR AMADEUS HOFFMAN]', 'Erckmann-Chatrian' : '[ERCKMANN-CHATRAIN]', 
               'Fiona Macleod' : '[FIONA MACLEOD]', 'Amelia B. Edwards' : '[AMELIA B. EDWARDS]', 'H. B. Marryatt' : '[H. B. MARRYATT]', 'Thomas Hardy' : '[THOMAS HARDY]', 
               'Montague Rhodes James' : '[MONTAGUE RHODES JAMES]', 'Fitz-James O\'Brien' : '[FITZ-JAMES O\'BRIEN', 'James Stephen' : '[JAMES STEPHEN]', 'Alfred Lord Tennyson' : '[ALFRED LORD TENNYSON]',
               'Amelia Edwards' : '[AMELIA EDWARDS]', 'Edward Bulwer-Lytton' : '[EDWARD BULWER-LYTTON]', 'Erckmann Chatrian' : '[ERCKMANN CHATRIAN]', 'Latifa al-Zayya' : '[LATIFA AL-ZAYYA]',
               'M. R. James' : '[M. R. JAMES]', 'Paul Brandis' : '[PAUL BRANDIS]', 'Brain Evenson' : '[BRAIN EVENSON]', 'Elliott O\'Donnell' : '[ELLIOTT O\'DONNELL]', 
               'Joseph, Sheridan Le Fanu' : '[JOSEPH, SHERIDAN LE FANU]', 'Edgar Allan Poe' : '[EDGAR ALLEN POE]', 'Bram Stoker' : '[BRAM STOKER]', 'Algernon Blackwood' :'[ALGERNON BLACKWOOD]',
               'Miles Klee' : '[MILES KLEE]', 'Nnedi Okorador' : '[NNEDI OKORADOR]', 'Sofia Samatar' : '[SOFIA SAMATAR]', 'Franz Kafka' : '[FRANZ KAFKA]', 'Laird Barron' : '[LAIRD BARRON]',
               'Nathan Ballingrud' : '[NATHAN BALLINGRUD]', 'Nellie Bly' : '[NELLIE BLY]', 'William Hop Hodgson' : '[WILLIAM HOP HODGSON]', 'Ambrose Bierce' : '[AMBROSE BIERCE]',
               'Kelly Link' : '[KELLY LINK]', 'Arthur Machen' : '[ARTHUR MACHEN]', 'George Sylvester Viereck' : '[GEORGE SYLVESTER VIERECK]', 'Robert Chambers' : '[ROBERT CHAMBERS]',
               'John Meade Falkner' : '[JOHN MEADE FALKNER]', 'Ann Radcliffe' : '[ANN RADCLIFFE]', 'Howard Lovecraft' : '[HOWARD LOVECRAFT]', 'Louis Stevenson' : '[LOUIS STEVENSON]',
               'Edith Birkhead' : '[EDITH BIRKHEAD]', 'Jeff Vandermeer' : '[JEFF VANDERMEER]', 'Henry James' : '[HENRY JAMES]', 'John William Polidori' : '[JOHN WILLIAM POLIDORI]',
               'Bob Holland' : '[BOB HOLLAND]', 'Oliver Onions' : '[OLIVER ONIONS]'}


# Initializing a GPT2 configuration
# configuration = GPT2Config()
# Initializing a model from the configuration
# model = GPT2Model(configuration)



SPECIAL_TOKENS = list({**literary_tokens, **subgenre_tokens, **author_tokens}.values())
# tokenizer = AutoTokenizer.from_pretrained("pranavpsv/gpt2-genre-story-generator", additional_special_tokens=SPECIAL_TOKENS)
# model = AutoModelWithLMHead.from_pretrained("pranavpsv/gpt2-genre-story-generator")
tokenizer = AutoTokenizer.from_pretrained("aadelucia/GPT2_medium_narrative_finetuned_large", additional_special_tokens=SPECIAL_TOKENS)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AutoModelWithLMHead.from_pretrained(last_checkpoint).to(device)
# Accessing the model configuration
# Set the number of special tokens in the model
model.resize_token_embeddings(len(tokenizer.get_vocab()))
configuration = model.config
print(configuration)

train_path = parent_directory + 'Data/Tokenized/merged_tokenized_train.txt';
test_path = parent_directory + 'Data/Tokenized/merged_tokenized_test.txt';
output_path = parent_directory + checkpoint_path

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)



/usr/local/lib/python3.6/dist-packages/transformers/modeling_auto.py:837: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


GPT2Config {
  "_name_or_path": "/content/drive/My Drive/CSCI 470 Project/Epic/Models/gpt2_large_horror_generator/checkpoint-18000",
  "_num_labels": 2,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0,
  "output_past": true,
  "predict_special_tokens": true,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "vocab_size": 50260
}



/usr/local/lib/python3.6/dist-packages/transformers/data/datasets/language_modeling.py:44: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
wandb.login()
# Optional: log both gradients and parameters
%env WANDB_WATCH=all
%env WANDB_PROJECT=Epic
%env TASK_NAME=gpt2_large_V1_continued_training_6

epochs = 3
batch_size= 10

training_args = TrainingArguments(
  run_name="gpt2_large_V1_continued_2",
  output_dir=output_path, #The output directory
  overwrite_output_dir=True, #overwrite the content of the output directory, set to true if continuing training
  num_train_epochs=epochs, # number of training epochs
  per_device_train_batch_size=batch_size, # batch size for training
  per_device_eval_batch_size=int(batch_size/2),  # batch size for evaluation
  eval_steps = 100, # Number of update steps between two evaluations.
  save_steps = 500, # after # steps model is saved
  warmup_steps=200,# number of warmup steps for learning rate scheduler
  evaluation_strategy="steps",
  logging_steps = 10,
  logging_dir=output_path+"/logs",
  do_predict=True,
  )

trainer = Trainer(
  model=model,
  args=training_args,
  data_collator=data_collator,
  train_dataset=train_dataset,
  eval_dataset=test_dataset
)
try:
  print("Train Path:\t", train_path)
  print("Test Path:\t", test_path)
  print("Output Directory Path:\t", output_path)
  trainer.train(last_checkpoint)

finally:
  print("Exiting Training")
  trainer.save_model(output_path+"/latest_model")
  tokenizer.save_pretrained(output_path+"/tokenizer")
  print(trainer.evaluate())
  print('All changes made in this colab session should now be visible in Drive.')


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


env: WANDB_WATCH=all
env: WANDB_PROJECT=Epic
env: TASK_NAME=gpt2_large_V1_continued_training_6
Train Path:	 /content/drive/My Drive/CSCI 470 Project/Epic/Data/Tokenized/merged_tokenized_train.txt
Test Path:	 /content/drive/My Drive/CSCI 470 Project/Epic/Data/Tokenized/merged_tokenized_test.txt
Output Directory Path:	 /content/drive/My Drive/CSCI 470 Project/Epic/Models/gpt2_large_horror_generator/


Step,Training Loss,Validation Loss
18100,2.428760,2.286101
18200,2.485565,2.285921
18300,2.357825,2.284501
18400,2.295966,2.284239
18500,2.472327,2.282013
18600,2.447705,2.281380
18700,2.517322,2.280214
18800,2.587439,2.278936
18900,2.462915,2.279916
19000,2.310107,2.277810


Exiting Training


{'eval_loss': 2.2501144409179688, 'epoch': 3.0}
All changes made in this colab session should now be visible in Drive.


In [ ]:

# from transformers import pipeline
# from glob import glob

# # Example of usage
# from transformers import pipeline

# story_gen = pipeline("text-generation", "pranavpsv/gpt2-genre-story-generator")
# print(story_gen("<BOS> <superhero> Batman"))

# generator = pipeline('text-generation',model=output_path+"/checkpoint-000", tokenizer=tokenizer ,config={'max_length':800})

# result = generator('Input Prompt')[0]['generated_text']